# Preparation of spike trains for TriDy/TopoSampling pipeline

This notebook is used to prepare the spike trains obtained from simulations for classification using a neighborhood-based stimulus classification pipeline (TriDy/TopoSampling pipeline; see [`../../classification`](../../classification)). As opposed to reliabiliy simulations (short simulations, but multiple seeds) we assume here simulation campaigns with only one (long) simulation of a single seed.

__Data preparation overview:__
- Neuron info (mc2 target): __neuron_info.pickle__ (and __.h5__, nut not required by pipeline)
- EXC spikes (mc2 target; EXC only): __raw_spikes_exc.npy__
- Stimulus train: __stim_stream.npy__ (and __time_windows.npy__, but not required by pipeline)
- Adjacency matrix (mc2 target; re-indexed): __connectivity.npz__

In [1]:
import numpy as np
import os
import sys
sys.path.append('../../../library')
from extract import run_extraction  # Requires bluepy kernel

In [2]:
syn_class = 'EXC'

output_dir_name = 'toposample_input'  # Default name
print(f'Output dir: "{output_dir_name}" ({syn_class} spike trains)')

Output dir: "toposample_input" (EXC spike trains)


In [3]:
# BlueConfig campaigns
# campaign_path = '/gpfs/bbp.cscs.ch/data/scratch/proj9/bisimplices/simulations/Toposample_O1v5-SONATA'  # Baseline
# campaign_path = '/gpfs/bbp.cscs.ch/data/scratch/proj9/bisimplices/simulations/Toposample_O1v5-SONATA_ConnRewired_mc2EE_Order1'
# campaign_path = '/gpfs/bbp.cscs.ch/data/scratch/proj9/bisimplices/simulations/Toposample_O1v5-SONATA_ConnRewired_mc2EE_Order2'
campaign_path = '/gpfs/bbp.cscs.ch/data/scratch/proj9/bisimplices/simulations/Toposample_O1v5-SONATA_ConnRewired_mc2EE_Order3'

sim_paths, output_dir = run_extraction(campaign_path, working_dir_name=output_dir_name, syn_class=syn_class)
num_sims = len(sim_paths)

/gpfs/bbp.cscs.ch/home/pokorny/BbpWorkflowKernel/lib/python3.9/site-packages/tables/file.py:411: UserWarning: a closed node found in the registry: ``/neuron_info/meta/values_block_3/meta/_i_table``
  warnings.warn("a closed node found in the registry: "
/gpfs/bbp.cscs.ch/ssd/apps/bsd/2022-01-10/stage_applications/install_gcc-11.2.0-skylake/py-numpy-1.19.5-gjm7kk/lib/python3.9/site-packages/numpy/lib/arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


INFO: 1 spike files written to "/gpfs/bbp.cscs.ch/data/scratch/proj9/bisimplices/bbp_workflow/e23aa485-8885-4e50-bd1f-9a9661463704/toposample_input"


In [4]:
# Check if only one sim as expected; in this case we just rename the file and we are done!
if num_sims == 1:
    src_file = os.path.join(output_dir, 'raw_spikes_exc_0.npy')
    tgt_file = os.path.join(output_dir, 'raw_spikes_exc.npy')
    os.rename(src_file, tgt_file)

    spk = np.load(tgt_file)
    stim = np.load(os.path.join(output_dir, 'stim_stream.npy'))
    print(f'{spk.shape[0]} spikes from {np.min(spk[:, 0]):.3f}ms to {np.max(spk[:, 0]):.3f}ms\n')
    print(f'{len(np.unique(stim))} patterns, {len(stim)} stimuli in total:')
    print('  ' + ', '.join([f'P{_sid}: {_cnt}'  for _sid, _cnt in zip(*np.unique(stim, return_counts=True))]))
else:
    print(f'WARNING: {num_sims} spike files in campaign! Merging required into a single file!')


3235133 spikes from 0.025ms to 159999.850ms

8 patterns, 800 stimuli in total:
  P0: 100, P1: 100, P2: 100, P3: 100, P4: 100, P5: 100, P6: 100, P7: 100
